# TimefnInvert.py 

This is the simplest time-series inversion method that can handle disconnected interferogram networks. The time-series inversion is driven by a model for deformation provided by the users. 

In [ ]:
cd /home/jovyan/work/notebooks/TimeSeries/GRFN/GIAnT

## 1. Recap

For our GRFN dataset, we have a good idea on when the eruptions occured. 

Here is a link related to the eruptions: https://disasters.nasa.gov/kilauea-hawaii-eruption-2018

This was confirmed by inspection of the SBAS time-series in the **plotts_notebook**.


## 2. How can we represent the deformation mathematically?

1. We noticed that the eruption occured sometime between Apr-23-2018 and May-5-2018.

2. Reading up on the event, we know that the eruption actually occured on May-3-2018.

3. We also noticed that the rate of displacement was different before and after the eruption. 

4. How can we capture this as a math function?

Here is one possible candidate?

$$d(t) = v * \left(t - t_0 \right) + a * \left( t - t_0 \right)^2 + D \cdot u \left(t - t_{erupt} \right) $$


How can we use GIAnT to estimate the parameters $v$, $a$ and $D$?

## 3. Updating userfn.py with Timefn

**Appendix B** of the GIAnT user manual provides details on how different math functions can be combined into a model. 

The list of functions that operate supported by GIAnT are shown below. These functions are evaluated on time elapsed in fractional years since the first acquisition.

<table>
    <thead>
        <tr>
            <td>Function name</td><td>Description</td>
        </tr>
    </thead>
    <tr>
        <td>Linear</td><td>Constant rate of deformation</td>
    </tr>
    <tr>
        <td>Polynomial</td><td>Polynomial of specified order</td>
    </tr>
    <tr>
        <td>Power</td><td>Time raised to specified order</td>
    </tr>
    <tr>
        <td>Exponential decay</td><td>For transient signals</td>
    </tr>
    <tr>
        <td>Logarithmic decay</td><td>For transient signals</td>
    </tr>
    <tr>
        <td>Step function </td><td>Modeling coseismic jumps</td>
    </tr>
    <tr>
        <td>B-splines</td><td>For transients, used with regularization</td>
    </tr>
    <tr>
        <td>Integrated B-splines</td><td>For transients, used with regularization</td>
    </tr>
</table>
        

Let's add the model representation shown above to **userfn.py**. To do this, we need to know the eruption time in fractional years since the first acquisition. Let's read the dates from **RAW-STACK.h5** and figure this out.

In [ ]:
import h5py
import datetime

fid = h5py.File('Stack/RAW-STACK.h5')
firstdate = fid['dates'][0]
fid.close()

print('First date: ', datetime.date.fromordinal(int(firstdate)))

Now that we have the dates, we can add the following to **userfn.py**

```python
def timedict():
    import datetime
    t_erupt = (datetime.date(2018,5,3) - datetime.date(2018,3,30)).days/365.25
    
    rep = [['POLY', [2], [0.]],
           ['STEP', [t_erupt]]]
    return rep
```

In [ ]:
#Look at userfn.py to confirm changes
!cat userfn.py

## 4. Run TimefnInvert.py

In [ ]:
!TimefnInvert.py

## 4. Looking at the outputs

In [ ]:
##List contents 
!ls -ltr 

In [ ]:
##List contents of Stack
!ls Stack/*

### Exploring contents of the HDF5 file from command line

In [ ]:
##Using HDF5 tools
!h5ls Stack/TS-PARAMS.h5

In [ ]:
##More verbose
!h5ls -v Stack/TS-PARAMS.h5

In [ ]:
##Using gdal
!gdalinfo Stack/TS-PARAMS.h5

## 5. Visualizing the estimated model

In [ ]:
import h5py

##Lets get the data
fid = h5py.File('Stack/TS-PARAMS.h5', 'r')


#Read model names
mname = fid['mName'][:]
print(mname)

#Load model
model = fid['parms'][:]

fid.close()

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.figure(figsize=(10,12))
for ii in range(4):
    plt.subplot(2,2,ii+1)
    plt.imshow(model[:,:,ii]);
    plt.colorbar();
    plt.title(mname[0])

### Lets move on to the plotts_notebook to look at the estimated time-series